<a href="https://colab.research.google.com/github/francescopatane96/RNA-seq_Pipeline/blob/main/DESeq2_DEG_from_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("vsn")

In [ ]:
install.packages("pheatmap")

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

In [ ]:
#--------------------------htseq-count_input---------------------------

In [ ]:
dir <- 'directory_path'

In [ ]:
sampleFiles <- grep(".count", list.files(dir),value=TRUE, fixed=TRUE)
coldata <- read.csv('descriptor.csv', header=TRUE, sep='\t')

In [ ]:
rownames(coldata) <- coldata$ID

In [ ]:
coldata$PT_ID <- as.factor(coldata$PT_ID)
levels(coldata$PT_ID)=c('PT_1', 'PT_2', 'PT_3', 'PT_4', 'PT_5')

In [ ]:
#add condition column($)
coldata$condition <- factor(coldata$MOL_MARKER)

In [ ]:
library("DESeq2")

In [ ]:
dds <- DESeqDataSetFromHTSeqCount(sampleTable = coldata,

                                  directory = dir,

                                  design = ~ condition)

In [ ]:
dds

In [ ]:
#-----------------------------pre-filtering-----------------------------

In [ ]:
keep <- rowSums(counts(dds)) >= 10          #the choosen value depends on experimental design
dds <- dds[keep,]
dds

In [ ]:
#-------------------------------DEG-------------------------------------

In [ ]:
dds <- DESeq(dds)
res <- results(dds)
res

In [ ]:
write.csv(res,'DEG.csv', row.names=TRUE)

In [ ]:
#------------------------------Analysis------------------------------------

In [ ]:
resOrdered <- res[order(res$pvalue),]
summary(res)

In [ ]:
sum(res$padj < 0.1, na.rm=TRUE)

In [ ]:
res05 <- results(dds, alpha=0.05)
summary(res05)

In [ ]:
sum(res05$padj < 0.05, na.rm=TRUE)

In [ ]:
#--------------------------------MA plots---------------------------------

In [ ]:
plotMA(res, ylim=c(-10,10))

In [ ]:
plotMA(resLFC, ylim=c(-6,6))

In [ ]:
idx <- identify(res$baseMean, res$log2FoldChange)
rownames(res)[idx]

In [ ]:
#---------------------------plot counts----------------------------

In [3]:
library("ggplot2")

In [ ]:
plotCounts(dds, gene=which.min(res$padj), intgroup="condition")

d <- plotCounts(dds, gene=which.min(res$padj), intgroup="condition", 
                returnData=TRUE)
                
ggplot(d, aes(x=condition, y=count)) + 
  geom_point(position=position_jitter(w=0.1,h=0)) + 
  scale_y_log10(breaks=c(25,100,400))

In [ ]:
#----------------------------------report--------------------------------

In [ ]:
mcols(res)$description

In [ ]:
write.csv(as.data.frame(resOrdered), 
          "report_ordered.csv")

In [ ]:
resSig <- subset(resOrdered, padj <= 0.05)

In [ ]:
write.csv(resSig, "report_p005.csv")

In [ ]:
#-------------------------Data transformation-------------------------

In [ ]:
vsd <- vst(dds, blind=FALSE)
rld <- rlog(dds, blind=FALSE)
head(assay(vsd), 3)

In [ ]:
write.csv(assay(vsd), "matrix_vsdlog.csv" )

In [ ]:
library("vsn")

In [ ]:
# this gives log2(n + 1)
ntd <- normTransform(dds)

meanSdPlot(assay(ntd))

meanSdPlot(assay(vsd))

meanSdPlot(assay(rld))

In [ ]:
#---------quality assessment, clustering and visualization-------------

In [ ]:
library("pheatmap")

In [ ]:
#vsd
select <- order(rowMeans(counts(dds,normalized=TRUE)),
                decreasing=TRUE)[1:20]
df <- as.data.frame(colData(dds)[,c("condition", "SAMPLE_ORIGIN")])
pheatmap(assay(vsd), cluster_rows=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)

In [ ]:
#clustering
pheatmap(assay(vsd), cluster_rows=F, show_rownames=F,
         cluster_cols=T, annotation_col=df)

In [ ]:
#sampletosample distances, distance matrix
sampleDists <- dist(t(assay(vsd)))

In [ ]:
#PCA
plotPCA(vsd, intgroup=c("condition"))

In [ ]:
plotPCA(vsd, intgroup=c("condition", "SAMPLE_ORIGIN"))

In [ ]:
pcaData <- plotPCA(vsd, intgroup=c("condition", "SAMPLE_ORIGIN"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=condition, shape=SAMPLE_ORIGIN)) +
    geom_point(size=2) +
    xlab(paste0("PC1: ",percentVar[1],"% variance")) +
    ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
    coord_fixed()

In [ ]:
pcaData <- plotPCA(vsd, intgroup=c("condition", "SAMPLE_ORIGIN"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=condition, shape=PT_ID)) +
  geom_point(size=2) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()

In [ ]:
#dispersionestimates
plotDispEsts(dds)